In [ ]:
import numpy as np
import time
import tonic
import tonic.transforms as transforms
import coreset
from source.utilities import subset, custom_sampler
from pathlib import Path
from datetime import datetime
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath('source', nargout=0)

### parameters

In [ ]:
# dataset parameters
dataset_name = 'POKERDVS' # name of dataset: POKERDVS -- NMNIST -- NCARS -- DVSGesture
download_dataset = True # downloads the datasets before parsing
first_saccade_only = False # specific for N-MNIST (3 saccades 100ms each)
subsample = 100 # take a sample of the dataset
spatial_histograms = True
K = 10
coresets = False
Np = 1000
t_res = 1000

# algorithm parameters
C = 5
maxNumIter = 1000

### generating dataset

In [ ]:
start_time = time.time()

if dataset_name == 'NCARS': # 304 x 240
    train_set = tonic.datasets.NCARS(save_to='./data', train=True, download=download_dataset)
    test_set = tonic.datasets.NCARS(save_to='./data', train=False, download=download_dataset)
if dataset_name == 'POKERDVS': # 35 x 35
    train_set = tonic.datasets.POKERDVS(save_to='./data', train=True, download=download_dataset)
    test_set = tonic.datasets.POKERDVS(save_to='./data', train=False, download=download_dataset)
elif dataset_name == "DVSGesture": # 128 x 128
    train_set = tonic.datasets.DVSGesture(save_to='./data', train=True, download=download_dataset)
    test_set = tonic.datasets.DVSGesture(save_to='./data', train=False, download=download_dataset)
elif dataset_name == 'NMNIST': # 34 x 34
    train_set = tonic.datasets.NMNIST(save_to='./data/nmnist', train=True, download=download_dataset, first_saccade_only=first_saccade_only)
    test_set = tonic.datasets.NMNIST(save_to='./data/nmnist', train=False, download=download_dataset, first_saccade_only=first_saccade_only)
    
x_index = train_set.ordering.find('x')
y_index = train_set.ordering.find('y')
t_index = train_set.ordering.find('t')

In [ ]:
from torch.utils.data import DataLoader
train_index = subset(train_set, subsample)
trainloader = DataLoader(train_set, sampler=custom_sampler(train_index), shuffle=False)

test_index = subset(test_set, 100)
testloader = DataLoader(test_set, sampler=custom_sampler(test_index), shuffle=False)

dirname = "./data/"+dataset_name.lower()+"_"+datetime.now().strftime("%Y_%m_%d-%I:%M:%S_%p")
Path(dirname).mkdir(parents=True, exist_ok=True)
for i, (events, target) in enumerate(trainloader):
    events = events.numpy().squeeze()
    data = np.vstack((events[:,t_index]/t_res, events[:,x_index], events[:,y_index])).T
    if coresets: data, _ = coreset.generate(data, Np)
    np.savetxt(dirname+"/"+dataset_name.lower()+"_%s.txt" % (i+1), data, delimiter=' ', fmt='%f')

In [ ]:
start_time = time.time()
M = len(trainloader)
centers = np.asarray(eng.jrmpc(dirname+"/"+dataset_name.lower()+'_%d.txt', M, C, 'maxNumIter', maxNumIter, 'gamma', 0.1)).T
print('--- %s seconds ---' % (time.time() - start_time))

In [ ]:
def predict(centers, data):
    labels = np.zeros(data.shape[0], dtype=int)
    for i, d in enumerate(data):
        labels[i] = np.argmin(np.linalg.norm(centers - d, axis=1)**2)
    return labels

In [ ]:
def create_histograms(dataloader, centers, C, dataset):
    x_index = dataset.ordering.find("x")
    y_index = dataset.ordering.find("y")
    t_index = dataset.ordering.find("t")
    X = []
    Y = np.zeros(len(dataloader))
    for i, (events, target) in enumerate(dataloader):
        events = events.numpy().squeeze()
        data = np.vstack((events[:,t_index]/t_res, events[:,x_index], events[:,y_index])).T
        Y_pred = predict(centers, data)
        Y[i] = target.numpy()[0]
        X.append(np.histogram(Y_pred, bins=np.arange(0, C))[0])
    return X, Y

def create_spatial_histograms(dataloader, centers, C, dataset, K):
    sensor_size = dataset.sensor_size
    x_index = dataset.ordering.find("x")
    y_index = dataset.ordering.find("y")
    t_index = dataset.ordering.find("t")
    X = []
    Y = np.zeros(len(dataloader))
    n_cells = (sensor_size[0] // K+1) * (sensor_size[1] // K+1)
    for i, (events, target) in enumerate(dataloader):
        events = events.numpy().squeeze()
        data = np.vstack((events[:,t_index]/t_res, events[:,x_index], events[:,y_index])).T
        Y_pred = predict(centers, data)
        Y[i] = target.numpy()[0]
        
        cells = []
        cell_index = 0
        for i in range(sensor_size[0] // K +1):
            for j in range(sensor_size[1] // K +1):
                xs = events[:,x_index]
                ys = events[:,y_index]
                selection = events[(xs >= i*K) & (xs < i*K+K) & (ys >= j*K) & (ys < j*K+K)]
                if len(selection) > 0:
                    cells.extend([cell_index] * len(selection))
                cell_index += 1
        
        hists = []
        for i in np.arange(n_cells):
            selection = (cells == i)
            if len(selection) > 0:
                hists.append(np.histogram(Y_pred[selection], bins=np.arange(0, C+1))[0])
            else:
                hists.append(np.zeros(C))
        X.append(np.concatenate(hists))
    return X, Y

### pytorch classification

In [ ]:
from source.logreg import LogisticRegression
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from sklearn import preprocessing
from datetime import datetime

start_time = time.time()

# creating histograms from hard clusters
if spatial_histograms:
    train_features, train_labels = create_spatial_histograms(trainloader, centers, C, train_set, K)
    test_features, test_labels = create_spatial_histograms(testloader, centers, C, test_set, K)
else:
    train_features, train_labels = create_histograms(trainloader, centers, C, train_set)
    test_features, test_labels = create_histograms(testloader, centers, C, test_set)

# scale features to 0 mean and 1 variance
scaler = preprocessing.StandardScaler().fit(train_features)
train_features = scaler.transform(train_features)
test_features = scaler.transform(test_features)

# creating dataloaders
training_dataset = TensorDataset(torch.Tensor(train_features),torch.Tensor(train_labels))
training_dataloader = DataLoader(training_dataset, batch_size=128)

test_dataset = TensorDataset(torch .Tensor(test_features),torch.Tensor(test_labels))
test_dataloader = DataLoader(test_dataset, batch_size=128)

# finding unique classes
classes = np.unique(test_labels)

# training pytorch logistic regression
logreg = LogisticRegression(train_features.shape[1], len(classes), epochs=200, lr=0.01, step_size=30, gamma=1, momentum=0, weight_decay=0)
logreg.fit(training_dataloader)
score = logreg.score(test_dataloader)*100

# print score
print(score)

# save results
p = Path('jrmpc')
p.mkdir(exist_ok=True)
date = datetime.now().strftime("%Y_%m_%d-%I:%M:%S_%p")
filename = dataset_name+'_'+str(score)+'_'+str(C)+'_sp-hist_'+str(spatial_histograms)+'_'+date
np.save(p/filename, score)

print('--- %s seconds ---' % (time.time() - start_time))